In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [14]:
# Load the data
df = pd.read_csv('labeled-data02.csv')

# Exclude examples labeled 'Done'
df_filtered = df[(df['label'] != 'Done') & (df['label'] != 'N')]

# Extract features and labels
# Only include features F3,F4,C3,Cz,C4,P3,P4
X = df_filtered.iloc[:, 2:9].values 
y = df_filtered['label'].values

In [15]:
# Specify the order of your classes
class_order = ['L', 'R']  

# Initialize and fit the LabelEncoder with specified classes
label_encoder = LabelEncoder()
label_encoder.fit(class_order)

# Transform labels to encoded format
y_encoded = label_encoder.transform(y)

In [16]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.1, random_state=42)

# Reshape the data for Conv1D layer
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [17]:
# X_train.shape[1] = number of features in data
# Model with 1 convolution and 1 dense layer
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())  # Flatten the output before the Dense layer
model.add(Dense(64, activation='relu'))  # Second layer is Dense with 64 neurons and ReLU activation
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification


In [18]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
25/25 [==============================] - 0s 6ms/step - loss: 0.6539 - accuracy: 0.6305 - val_loss: 0.6094 - val_accuracy: 0.6742
Epoch 2/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5926 - accuracy: 0.6898 - val_loss: 0.5761 - val_accuracy: 0.6742
Epoch 3/10
25/25 [==============================] - 0s 2ms/step - loss: 0.5707 - accuracy: 0.6961 - val_loss: 0.5727 - val_accuracy: 0.7079
Epoch 4/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5574 - accuracy: 0.7125 - val_loss: 0.5536 - val_accuracy: 0.7079
Epoch 5/10
25/25 [==============================] - 0s 2ms/step - loss: 0.5430 - accuracy: 0.7188 - val_loss: 0.5561 - val_accuracy: 0.7191
Epoch 6/10
25/25 [==============================] - 0s 2ms/step - loss: 0.5352 - accuracy: 0.7314 - val_loss: 0.5891 - val_accuracy: 0.7079
Epoch 7/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5320 - accuracy: 0.7364 - val_loss: 0.5711 - val_accuracy: 0.6966
Epoch 8/10
25/25 [==

In [20]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Test Accuracy: {accuracy}')

4/4 [==============================] - 0s 984us/step
Test Accuracy: 0.7244897959183674
